In [17]:
import torch_geometric
import networkx as nx
from typing import Optional, List, Any
from torch_geometric.utils.convert import from_networkx
from heapq import heapify

In [10]:
DG = nx.DiGraph()

In [13]:
def get_parent(i: int) -> Optional[int]:
    if i <= 0:
        return None
    return (i - 1) // 2

def is_heap(l) -> bool:
    for (i, e) in enumerate(l):
        parent = get_parent(i)
        if parent is not None and l[parent] > e:
            return False
    return True

In [14]:
is_heap([5, 1, 4])

False

In [15]:
x = [5, 2, 5, 2]
assert not is_heap(x)

In [18]:
heapify(x)
x

[2, 2, 5, 5]

In [19]:
assert is_heap(x)